In [1]:
import sys 
sys.path.append('../../')
from config.HIVESERVER2_token import *

import pandas as pd
import pendulum
from tqdm import trange
from jinja2 import Template
from pyutil.pysql.sql import SQLClient

client = SQLClient('mixx')

In [38]:
DS = pendulum.Date(2023,6,20)
temp = Template(open("sql/0.0.3.v2.sql",'r').read())
for i in trange(30):
    ds = DS.add(days=-i).to_date_string()
    [client.execute_trino_query(q) for q in temp.render(ds = ds).split(';')]

100%|██████████| 30/30 [20:57<00:00, 41.92s/it]


In [ ]:
# originaldt = pendulum.Date(2023,6,17)
# extractUDF = F.udf(lambda x: '\x02'.join([v for v in x.split('\x02') if v.split('\x03')[0] in feature_name]), T.StringType())

# for i in trange(0,30):
#     dt = originaldt.add(days=i)
#     dtstr = dt.to_date_string()
#     start, end = dt.add(days=1).to_date_string(), dt.add(days=7).to_date_string()

#     if end >= pendulum.today().to_date_string():
#         break
    
#     sample = (
#         spark
#         .sql(f"select * from 10048_ml_dw.bundle_price_role_common_feature_dd where dt = '{dtstr}' and  feature_group = 'attribute' ")
#         .withColumn('subf', F.explode(F.split(F.col('feature'),'\x02')))
#         .withColumn('fname', F.substring_index(F.col('subf'),'\x03', 1))
#         .where(F.col('fname').isin(feature_name))
#         .groupBy('acc_id', 'role_id', 'dt')
#         .agg( F.concat_ws('\x02', F.collect_set('subf')).alias('feature'))
#         .withColumn('dt', F.lit(dtstr))
#         .withColumn('grp', F.lit('common'))
#         .select('role_id','acc_id','feature','grp','dt')
#     )
    
#     sample.write.mode("overwrite").insertInto('ml_tmp_db.wgame_bundle_dynamic_pricing_feature_res_ratityv2', overwrite=True)

#     sample.unpersist()


In [13]:
client.reset()
DS = pendulum.Date(2023,6,20)
for i in trange(4):
    ds = DS.add(days=-i).to_date_string()
    temp = Template(open("sql/res_feature.sql",'r').read())
    [client.execute_trino_query(q) for q in temp.render(ds = ds).split(';')]


100%|██████████| 4/4 [01:27<00:00, 21.99s/it]


In [2]:
q = '''
SELECT 
    a.*
    , r.svr_region
    , r.language
    , r.region
    , b.feature AS role_feature
FROM (
    SELECT *
    FROM hive.ml_tmp_db.wgame_bunlde_sequence_sample 
    WHERE dt = '{ds}'
) as a
INNER JOIN kudu."10048_bi_ods".total_role_attr_all AS r
ON a.open_id = r.acc_id 
    AND a.user_id = r.role_id 
INNER JOIN (
    SELECT acc_id, role_id, ARRAY_AGG(feature) AS feature, dt
    FROM hive.ml_tmp_db.wgame_bundle_dynamic_pricing_feature_res_ratityv2
    WHERE feature_group in ('res_ratity','common') AND dt = '{ds}'
    GROUP BY acc_id, role_id, dt
) b
ON a.open_id = b.acc_id 
    AND a.user_id = b.role_id 
    AND a.dt = b.dt
'''


data = []
DS = pendulum.Date(2023,6,20)

for i in trange(30):
    ds = DS.add(days=-i).to_date_string()
    print(ds)
    client.execute_trino_query(q.format(ds = ds))
    data.append(client.to_pandas())

  0%|          | 0/30 [00:00<?, ?it/s]

2023-06-20


  3%|▎         | 1/30 [00:56<27:18, 56.51s/it]

2023-06-19


  7%|▋         | 2/30 [01:55<27:04, 58.01s/it]

...

 93%|█████████▎| 28/30 [32:52<01:51, 55.97s/it]

2023-05-23


 97%|█████████▋| 29/30 [34:01<00:59, 59.73s/it]

2023-05-22


100%|██████████| 30/30 [34:59<00:00, 69.98s/it]


In [2]:
cat_feature = [
    'svr_region'
]

common_feature_boolean = [
    "role_create_os_android",
    "role_create_os_ios",
    "if_accounts_first_role",
    "if_pruchase_active_in_1day",
    "if_pruchase_active_in_30day",
    "if_pruchase_active_in_3day",
    "if_pruchase_active_in_7day",
    "is_acc_transfer_from_old",
    "is_acc_transfer_to_new",
    "is_role_transfer_from_old",
    "is_role_transfer_to_new",
    "is_valid",

    "role_type_fraud",
    "role_type_no_install_info",
]

common_feature_numeric = [
    "acc_first_purchase_price_usd",
    "acc_role_cnt",
    "acc_total_charge_cnt",
    "acc_total_charge_sum_usd",
    "acc_total_refund_cnt",
    "acc_total_refund_sum_usd",
    "bundle_pop_buy_count_last_day",
    "bundle_pop_buy_count_last_day_campaign_stage_trigger",
    "bundle_pop_buy_count_last_day_tech_unlock_trigger",
    "bundle_pop_buy_count_last_week",
    "bundle_pop_buy_count_last_week_adjust",
    "bundle_pop_buy_max_last_day",
    "bundle_pop_buy_max_last_week",
    "bundle_pop_buy_max_last_week_adjust",
    "bundle_pop_buy_max_price_campaign_stage_trigger",
    "bundle_pop_buy_max_price_campaign_stage_trigger_commax",
    "bundle_pop_buy_max_price_tech_unlock_trigger",
    "bundle_pop_buy_max_price_tech_unlock_trigger_commax",
    "bundle_pop_buy_sum_last_day",
    "bundle_pop_buy_sum_last_week",
    "bundle_pop_buy_sum_last_week_adjust",
    "bundle_pop_count_campaign_stage_trigger_9999",
    "bundle_pop_count_last_day",
    "bundle_pop_count_last_day_campaign_stage_trigger",
    "bundle_pop_count_last_day_tech_unlock_trigger",
    "bundle_pop_count_last_week",
    "bundle_pop_count_last_week_adjust",
    "bundle_pop_count_tech_unlock_trigger_9999",

    "level",
    "max_role_level",
    "money_flow_in_cout_last_day",
    "money_flow_in_cout_last_week",
    "money_flow_in_cout_last_week_adjust",
    "money_flow_out_cout_last_day",
    "money_flow_out_cout_last_week",
    "money_flow_out_cout_last_week_adjust",
    "purchase_price_1499_last_week",
    "purchase_price_1499_last_week_adjust",
    "purchase_price_1999_last_week",
    "purchase_price_1999_last_week_adjust",
    "purchase_price_2999_last_week",
    "purchase_price_2999_last_week_adjust",
    "purchase_price_4999_last_week",
    "purchase_price_4999_last_week_adjust",
    "purchase_price_499_last_week",
    "purchase_price_499_last_week_adjust",
    "purchase_price_9999_last_week",
    "purchase_price_9999_last_week_adjust",
    "purchase_price_999_last_week",
    "purchase_price_999_last_week_adjust",
    "purchase_price_max",
    "purchase_price_max_last_3days",
    "purchase_price_max_last_day",
    "purchase_price_max_last_month",
    "purchase_price_max_last_week",
    "purchase_price_max_last_week_adjust",
    "purchase_price_sum_last_3days",
    "purchase_price_sum_last_day",
    "purchase_price_sum_last_month",
    "purchase_price_sum_last_week",
    "purchase_price_sum_last_week_adjust",

    
    "role_first_purchase_price_usd",
    "role_last_login_duration",
    "role_last_purchase_price_usd",
    "role_lifespan",
    "role_login_last_now",
    "role_login_secondlast_last",
    "role_login_secondlast_now",
    "role_logout_last_now",
    "role_no",
    "role_purchase_first_last",
    "role_purchase_first_now",
    "role_purchase_last_now",
    "role_total_charge_cnt",
    "role_total_charge_sum_usd",
    "role_total_refund_cnt",
    "role_total_refund_sum_usd",
    
    "vip_level",
]

common_feature_ratio = [
    "purchase_price_sum_max_ratio",
    "purchase_price_sum_max_ratio_com2",


    'resource_rarity',
    'speedup_rarity',
    'top_key_rarity',
    'officer_bar_ticket_rarity',
    'common_equipment_exp_consume_levelup_1_diff_num_relatively',
    'common_equipment_exp_consume_blueprint_1_diff_num_relatively',
    'common_equipment_exp_rarity',
    'airforce_equipment_exp_consume_levelup_1_diff_num_relatively',
    'airforce_equipment_exp_consume_blueprint_1_diff_num_relatively',
    'airforce_equipment_exp_rarity',
    'ammunition_consume_levelup_1_diff_num_relatively',
    'ammunition_consume_levelup_10_diff_num_relatively',
    'ammunition_consume_blueprint_1_diff_num_relatively',
    'ammunition_consume_blueprint_10_diff_num_relatively',
    'ammunition_rarity',
    'airforce_ammunition_consume_levelup_1_diff_num_relatively',
    'airforce_ammunition_consume_levelup_10_diff_num_relatively',
    'airforce_ammunition_consume_blueprint_1_diff_num_relatively',
    'airforce_ammunition_consume_blueprint_10_diff_num_relatively',
    'airforce_ammunition_rarity',
    'elements_consume_blueprint_1_diff_num_relatively',
    'elements_consume_blueprint_10_diff_num_relatively',
    'elements_consume_reform_diff_num_relatively',
    'elements_rarity',
    'airforce_elements_consume_blueprint_1_diff_num_relatively',
    'airforce_elements_consume_blueprint_10_diff_num_relatively',
    'airforce_elements_consume_reform_diff_num_relatively',
    'airforce_elements_rarity',
    'replacement_wrench_consume_mechanic_1_diff_num_relatively',
    'replacement_wrench_rarity',
    'officer_exp_consume_lvlup_1_diff_num_relatively',
    'officer_exp_rarity',
    'component_consume_diff_num_relatively',
    'chest_choice_replacement_rarity',

    "buy_rate_last_day",
    "buy_rate_last_day_campaign_stage_trigger",
    "buy_rate_last_day_tech_unlock_trigger",
    "buy_rate_last_week",
]

In [3]:
dataset = pd.concat(data)

In [3]:
# dataset.to_parquet('../../data/raw/0.0.3.pqt', compression='gzip')
dataset = pd.read_parquet('../../data/raw/0.0.3.pqt')

In [4]:
dataset.shape

(588161, 10)

In [8]:
d1 = dataset\
    .assign(
        role_feature = lambda x : x.role_feature.apply(eval)
    )\
    .assign(
    feature0 = lambda x : x.feature.apply(lambda y: {v.split('\x03')[0]:v.split('\x03')[1].split('\x04') for v in y.split('\x02')} if y is not None else {}),
    feature1 = lambda x : x.role_feature.apply(
        lambda x: {
            v.split('\x03')[0]:v.split('\x03')[1].split('\x04') 
            for y in x for v in y.split('\x02') 
        }
    ),
)

d1 = d1.assign(
    ** {
        x : d1.feature1.apply(lambda y: float(y.get(x,[None])[0]))
        for x in common_feature_numeric
    }
)

In [10]:
datasetv2 = (
    dataset
    .assign(
        role_feature = lambda x : x.role_feature.apply(lambda x : {v.split('\x03')[0]:v.split('\x03')[1] for _ in eval(x) for v in _.split('\x02')}),
        bundle_feature = lambda x : x.feature.apply(lambda y: {v.split('\x03')[0]:v.split('\x03')[1].split('\x04') for v in y.split('\x02')} if y is not None else {}),
    )
    .drop(['feature'], axis = 1)
)

In [31]:
from scipy.stats import boxcox
import numpy as np

def value_to_float(x):
    if type(x) == float or type(x) == int:
        return x
    if 'K' in x:
        if len(x) > 1:
            return float(x.replace('K', '')) * 1e3
        return 1000.0
    if 'M' in x:
        if len(x) > 1:
            return float(x.replace('M', '')) * 1e6
        return 1000000.0
    if 'B' in x:
        return float(x.replace('B', '')) * 1e9
    else:
        return float(x)

boxcox_vocab = dict()
logbucket_vocab = dict()

for x in common_feature_numeric:
    array = datasetv2.role_feature.apply(lambda v : value_to_float(v.get(x)))
    array = array.to_numpy()
    c = array.min()
    c = abs(c) + 1
    _, l = boxcox(array + c, None, None) 
    boxcox_vocab[x] = (l,c)

    minv, maxv = array.min() + c, array.max() + c

    logbucket_vocab[x] = np.exp(np.log(maxv)/20)

d:\Anaconda\lib\site-packages\numpy\core\_methods.py:233: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
d:\Anaconda\lib\site-packages\numpy\core\_methods.py:244: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)


In [35]:
sik = {
    'boxcox_vocab' : boxcox_vocab,
    'logbucket_vocab' : logbucket_vocab
}

import json
json.dump(sik, open('0.0.3.numericvocab.json','w'))

In [36]:
datasetv2.to_parquet('../../data/raw/0.0.3.v2.pqt', compression='gzip')
